In [160]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# 3.1. Dataset Preprocessing

## Understanding the Dataset

In [161]:
# Read the dataset
dataset = pd.read_csv('diabetes.csv')
dataset.head()

Pregnancies  Glucose  BloodPressure  SkinThickness  Insulin   BMI  \
0            6      148             72           35.0        0  33.6   
1            1       85             66           29.0        0  26.6   
2            8      183             64            0.0        0  23.3   
3            1       89             66           23.0       94  28.1   
4            0      137             40           35.0      168  43.1   

   DiabetesPedigreeFunction  Age  Outcome  
0                     0.627   50        1  
1                     0.351   31        0  
2                     0.672   32        1  
3                     0.167   21        0  
4                     2.288   33        1

In [162]:
print('number of samples in the dataset',dataset.shape[0])
print('number of features in the dataset',dataset.shape[1]-1)

number of samples in the dataset 768
number of features in the dataset 8


In [163]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               768 non-null    int64  
 1   Glucose                   768 non-null    int64  
 2   BloodPressure             768 non-null    int64  
 3   SkinThickness             762 non-null    float64
 4   Insulin                   768 non-null    int64  
 5   BMI                       763 non-null    float64
 6   DiabetesPedigreeFunction  768 non-null    float64
 7   Age                       768 non-null    int64  
 8   Outcome                   768 non-null    int64  
dtypes: float64(3), int64(6)
memory usage: 54.1 KB


## Handling missing values in the dataset
Missing values represent incomplete information in a dataset. Removing null values ensures that the dataset is complete and ready for analysis or modeling. It prevents errors in machine learning models, which typically cannot handle missing data. There are various techniques to handle missing values in a dataset:


1.   Deleting rows with missing values
2.   Imputation techniques: Replacing the missong value with the mean, mode or median of the non-missing values in the respective feature.

choose one of the above methods to handle missing values in the dataset.



In [164]:
# TODO: Print the count of null values in each column
print("Count of null values in each column before removing:")
for col_name, col_series in dataset.items():
    null_ct =  col_series.isna().sum()
    print (f"{col_name} - {null_ct}")

# for index, row in dataset.iterrows():
#     null_ct = row.isna().sum()
#     if null_ct >0:
#         print (f'{index}-{null_ct}')

# TODO: Implement a method to remove rows with null values
#Dropping rows with null values
dataset.dropna(inplace=True)

# Step 2: Verify that null values have been removed by printing the count again
print("\nCount of null values in each column after removing:")
for col_name, col_series in dataset.items():
    null_ct = col_series.isna().sum()
    print (f"{col_name} - {null_ct}")


Count of null values in each column before removing:
Pregnancies - 0
Glucose - 0
BloodPressure - 0
SkinThickness - 6
Insulin - 0
BMI - 5
DiabetesPedigreeFunction - 0
Age - 0
Outcome - 0

Count of null values in each column after removing:
Pregnancies - 0
Glucose - 0
BloodPressure - 0
SkinThickness - 0
Insulin - 0
BMI - 0
DiabetesPedigreeFunction - 0
Age - 0
Outcome - 0


## Handling outliers in the dataset
An outlier is a data point that differs significantly from the majority of other points in a dataset. It is an unusually high or low value. In Particular,The SVM algorith is sensitive to outliers.The sensitivity of SVM to outliers stems from its reliance on support vectors to define the optimal hyperplane. Thus, outliers can substantially skew the positioning of the hyperplane. there are various methods to handle outliers. In this exercise, we want to first utilize box-plot based methods to visualize the data and identify the outliers. Then we will use IQR to remove the outliers. in the following paragraphs, we will explain these methods in detail.




**Understanding the Boxplot structure:**
A boxplot summarizes the distribution of data by showing the minimum, first quartile (Q1), median (Q2), third quartile (Q3), and maximum values. It provides a clear visualization of the spread and skewness of the data, and any points lying outside the “whiskers” are considered potential outliers.


Lets display the boxplots for features in the dataset and identify the outliers:

In [165]:
X = dataset.iloc[:,:-1]

# Set up the subplot grid (4 columns, 2 rows)
fig, axes = plt.subplots(2, 4, figsize=(16, 8))

# Flatten the axes array to make indexing easier
axes = axes.flatten()

# Plot a boxplot for each feature in the dataset (excluding the last column)
for i, column in enumerate(X.columns):
    sns.boxplot(x=X[column], ax=axes[i])
    axes[i].set_title(f'Boxplot for {column}')

# Adjust layout
plt.tight_layout()
plt.show()

/Users/betterbrambola/miniconda3/envs/mymlenv/lib/python3.10/site-packages/seaborn/categorical.py:632: FutureWarning: SeriesGroupBy.grouper is deprecated and will be removed in a future version of pandas.
  positions = grouped.grouper.result_index.to_numpy(dtype=float)
/Users/betterbrambola/miniconda3/envs/mymlenv/lib/python3.10/site-packages/seaborn/categorical.py:632: FutureWarning: SeriesGroupBy.grouper is deprecated and will be removed in a future version of pandas.
  positions = grouped.grouper.result_index.to_numpy(dtype=float)
/Users/betterbrambola/miniconda3/envs/mymlenv/lib/python3.10/site-packages/seaborn/categorical.py:632: FutureWarning: SeriesGroupBy.grouper is deprecated and will be removed in a future version of pandas.
  positions = grouped.grouper.result_index.to_numpy(dtype=float)
/Users/betterbrambola/miniconda3/envs/mymlenv/lib/python3.10/site-packages/seaborn/categorical.py:632: FutureWarning: SeriesGroupBy.grouper is deprecated and will be removed in a future vers

This is how we can remove the outliers:
first  

1.   calculate the Q1 (25th percentile) and Q3 (75th percentile) for each column.
2.   compute the Interquartile Range (IQR). IQR (Interquartile Range) is the difference between the 75th percentile (Q3) and the 25th percentile (Q1) of a dataset, representing the range where the middle 50% of the data lies.

3.   Using the IQR, define lower and upper bounds beyond which data points are considered outliers.





In [166]:
# TODO: Calculate Q1 and Q3 for each column
q1_means, q3_means = [], []
for i, column in enumerate(X.columns):
    q1_means.append(X[column].quantile(0.25))
    q3_means.append(X[column].quantile(0.75))
    print(f'{column}    -  q1={q1_means[i]}, q3={q3_means[i]}')
print(76*'-')

# TODO: Calculate the lower and upper bounds for each column
l_bounds, u_bounds = [], []
for i, column in enumerate(X.columns):
    q1, q3 = q1_means[i], q3_means[i]
    iqr = q3 - q1
    l_bd = q1 - 1.5*iqr
    u_bd = q3 + 1.5*iqr
    l_bounds.append(l_bd)
    u_bounds.append(u_bd)
    print(f'{column}  - IQR={iqr}, l_bound={l_bd}, u_bound={u_bd}')
print(76*'-')

# TODO: Keep only rows where all columns are within bounds
# for idx, row in X.iterrows():
#     drop_idxs = []
#     for i in range (X.shape[1]):
#         print()
#         if (row[i]<l_bounds[i] or row[i]>u_bounds[i]):
#             drop_idxs.append(idx)
# print(drop_idxs)
# filtered_data = dataset.drop(drop_idxs)
filtered_data = dataset
for i, (col_name, col_series) in enumerate(X.items()):
    filtered_data = filtered_data[(col_series>=l_bounds[i]) & (col_series<=u_bounds[i])]
    print (filtered_data.shape)
#    condition = condition and col_series >=l_bounds[i] and col_series <= u_bounds[i] 

print (filtered_data.shape)




print(76*'-')


Pregnancies    -  q1=1.0, q3=6.0
Glucose    -  q1=99.0, q3=140.0
BloodPressure    -  q1=62.0, q3=80.0
SkinThickness    -  q1=0.0, q3=32.0
Insulin    -  q1=0.0, q3=128.0
BMI    -  q1=27.3, q3=36.5
DiabetesPedigreeFunction    -  q1=0.244, q3=0.626
Age    -  q1=24.0, q3=41.0
----------------------------------------------------------------------------
Pregnancies  - IQR=5.0, l_bound=-6.5, u_bound=13.5
Glucose  - IQR=41.0, l_bound=37.5, u_bound=201.5
BloodPressure  - IQR=18.0, l_bound=35.0, u_bound=107.0
SkinThickness  - IQR=32.0, l_bound=-48.0, u_bound=80.0
Insulin  - IQR=128.0, l_bound=-192.0, u_bound=320.0
BMI  - IQR=9.2, l_bound=13.500000000000002, u_bound=50.3
DiabetesPedigreeFunction  - IQR=0.382, l_bound=-0.32899999999999996, u_bound=1.1989999999999998
Age  - IQR=17.0, l_bound=-1.5, u_bound=66.5
----------------------------------------------------------------------------
(753, 9)
(748, 9)
(703, 9)
(702, 9)
(669, 9)
(661, 9)
(637, 9)
(630, 9)
(630, 9)
---------------------------------

/var/folders/2v/n05wnxtx3gj216g2xdy7xngw0000gn/T/ipykernel_28638/1464466132.py:32: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  filtered_data = filtered_data[(col_series>=l_bounds[i]) & (col_series<=u_bounds[i])]
/var/folders/2v/n05wnxtx3gj216g2xdy7xngw0000gn/T/ipykernel_28638/1464466132.py:32: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  filtered_data = filtered_data[(col_series>=l_bounds[i]) & (col_series<=u_bounds[i])]
/var/folders/2v/n05wnxtx3gj216g2xdy7xngw0000gn/T/ipykernel_28638/1464466132.py:32: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  filtered_data = filtered_data[(col_series>=l_bounds[i]) & (col_series<=u_bounds[i])]
/var/folders/2v/n05wnxtx3gj216g2xdy7xngw0000gn/T/ipykernel_28638/1464466132.py:32: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  filtered_data = filtered_data[(col_series>=l_bounds[i]) & (col_series<=u_bounds[i])]
/var/folders/2v/

## Standardization
Standardize the features values to have a mean of zero and standard deviation of 1. This ensures that all features contribute equally to the model

In [167]:
# X = dataset.iloc[:,:-1]
# Y = dataset.iloc[:,:-1]

X = filtered_data.iloc[:,:-1]
Y = filtered_data.iloc[:,-1]

In [168]:
from sklearn.preprocessing import StandardScaler

# TODO: Standardize feature values:
scaler = StandardScaler()
scaled_data = scaler.fit_transform(X)



## Split the data into train and test

In [169]:
from sklearn.model_selection import train_test_split

# TODO: Split the dataset into training and testing sets.
# Use 80% of the data for training and 20% for testing.
x_train, x_test, y_train, y_test = train_test_split(X, Y, train_size=0.8, shuffle=True)

print (x_train.shape)
print (y_train.shape)
print (x_test.shape)
print (y_test.shape)

(504, 8)
(504,)
(126, 8)
(126,)


# 3.2. Train a linear svm and report its accuracy

In [172]:
from sklearn import svm
from sklearn.metrics import accuracy_score

c = 1.0
for c in [0.1, 1.0, 5, 10, 100, 1000]:
    #TODO: Fit a linear svm on train data
    model = svm.SVC(kernel='linear', C = c)
    model.fit(x_train, y_train)

    # TODO: Report the Accuracy on Train data
    train_pred = model.predict(x_train)
    train_acc = accuracy_score(y_train, train_pred)
    print (f'C={c} --- Train Accuracy -{train_acc}')

    # TODO: Report the Accuracy on Test data
    test_pred = model.predict(x_test)
    test_acc = accuracy_score(y_test, test_pred)
    print (f'C={c} --- Test Accuracy -{test_acc}')


C=0.1 --- Train Accuracy -0.7996031746031746
C=0.1 --- Test Accuracy -0.7222222222222222
C=1.0 --- Train Accuracy -0.8075396825396826
C=1.0 --- Test Accuracy -0.7301587301587301
C=5 --- Train Accuracy -0.8075396825396826
C=5 --- Test Accuracy -0.7222222222222222
C=10 --- Train Accuracy -0.8075396825396826
C=10 --- Test Accuracy -0.7222222222222222
C=100 --- Train Accuracy -0.8095238095238095
C=100 --- Test Accuracy -0.7222222222222222
C=1000 --- Train Accuracy -0.7976190476190477
C=1000 --- Test Accuracy -0.7222222222222222


# 3.3. Use nonlinear kernels and report model accuracy


In [173]:
#TODO: Fit a Non-linear svm on train data
c = 1.0
for c in [0.1, 1.0, 5, 10, 100, 1000]:
    model = svm.SVC(kernel='rbf', C = c, gamma='scale')
    model.fit(x_train, y_train)

    # TODO: Report the Accuracy on Train data
    train_pred = model.predict(x_train)
    train_acc = accuracy_score(y_train, train_pred)
    print (f'C={c} --- Train Accuracy -{train_acc}')

    # TODO: Report the Accuracy on Test data
    test_pred = model.predict(x_test)
    test_acc = accuracy_score(y_test, test_pred)
    print (f'C={c} --- Test Accuracy -{test_acc}')


C=0.1 --- Train Accuracy -0.7023809523809523
C=0.1 --- Test Accuracy -0.626984126984127
C=1.0 --- Train Accuracy -0.7896825396825397
C=1.0 --- Test Accuracy -0.7222222222222222
C=5 --- Train Accuracy -0.7996031746031746
C=5 --- Test Accuracy -0.7142857142857143
C=10 --- Train Accuracy -0.8015873015873016
C=10 --- Test Accuracy -0.7142857142857143
C=100 --- Train Accuracy -0.8273809523809523
C=100 --- Test Accuracy -0.7063492063492064
C=1000 --- Train Accuracy -0.8531746031746031
C=1000 --- Test Accuracy -0.6904761904761905


Question: Did nonlinear kernels improve accuracy?

We didn't observe significant bump in the accuracies for the rbf SVM classifiers when compared to the linear ones. We'd tried with the following regularization constants - [0.1, 1.0, 5, 10, 100, 1000]

# 3.4. **Next Mile:** Find the best Hyperparameters automatically
you can use Grid Search in scikit learn to explore different hyperparameter combinations and find the best ones. Let's implemenet it step by step!

In [13]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'C': [0.1, 1, 10, 100],
    'kernel': ['rbf', 'linear', 'poly', 'sigmoid']
}

In [14]:
# TODO: Initialize the SVM classifier

# TODO: Initialize GridSearchCV with cross-validation

# TODO: Fit grid search

# TODO: Print the best parameters and best score

# TODO: Use the best estimator to make predictions

# TODO: Report the Accuracy on Train data

# TODO: Report the Accuracy on Test data